# Day 3: Mull It Over

### Part 1
"Our computers are having issues, so I have no idea if we have any Chief Historians in stock! You're welcome to check the warehouse, though," says the mildly flustered shopkeeper at the North Pole Toboggan Rental Shop. The Historians head out to take a look.

The shopkeeper turns to you. "Any chance you can see why our computers are having issues again?"

The computer appears to be trying to run a program, but its memory (your puzzle input) is corrupted. All of the instructions have been jumbled up!

It seems like the goal of the program is just to multiply some numbers. It does that with instructions like mul(X,Y), where X and Y are each 1-3 digit numbers. For instance, mul(44,46) multiplies 44 by 46 to get a result of 2024. Similarly, mul(123,4) would multiply 123 by 4.

However, because the program's memory has been corrupted, there are also many invalid characters that should be ignored, even if they look like part of a mul instruction. Sequences like mul(4*, mul(6,9!, ?(12,34), or mul ( 2 , 4 ) do nothing.

For example, consider the following section of corrupted memory:

`xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))`

Only the four highlighted sections are real mul instructions. Adding up the result of each instruction produces `161 (2*4 + 5*5 + 11*8 + 8*5)`.

Scan the corrupted memory for uncorrupted mul instructions. What do you get if you add up all of the results of the multiplications?




In [1]:
import com.psyndicate.aoc.readResourceAsText
import kotlin.test.assertEquals

val testInput = """xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"""

sealed class Operator {
    data class Mul(val a: Int, val b: Int) : Operator()
    data object Do : Operator()
    data object Dont : Operator()
}

fun parseOps(input: String): List<Operator> = buildList {
    val mulRegex = "mul\\((\\d{1,3}),(\\d{1,3})\\)|do\\(\\)|don\'t\\(\\)".toRegex()
    var index = 0
    while (index < input.length) {
        mulRegex.matchAt(input, index)?.let {
            val op = when {
                it.value == "do()" -> Operator.Do
                it.value == "don\'t()" -> Operator.Dont
                it.value.startsWith("mul(") -> {
                    val a = it.groups[1]?.value
                    val b = it.groups[2]?.value
                    if (!a.isNullOrBlank() && !b.isNullOrBlank()) {
                        Operator.Mul(a.toInt(), b.toInt())
                    } else {
                        null
                    }
                }
                else -> null
            }

            if (op != null) {
                add(op)
                index += it.value.length
            } else {
                index++
            }
        } ?: index++
    }
}

fun calculatePart1(input: String): Int = parseOps(input)
    .filterIsInstance<Operator.Mul>()
    .sumOf { mul -> mul.a * mul.b }

val testInputResult = calculatePart1(testInput)
println("Test Part 1: ${testInputResult}")
assertEquals(161, testInputResult)

val puzzleInput = readResourceAsText("/day3/input.txt")
val part1Result = calculatePart1(puzzleInput)
println("Part 1: ${part1Result}")
assertEquals(179571322, part1Result)


Test Part 1: 161
Part 1: 179571322


### Part 2

As you scan through the corrupted memory, you notice that some of the conditional statements are also still intact. If you handle some of the uncorrupted conditional statements in the program, you might be able to get an even more accurate result.

There are two new instructions you'll need to handle:

The do() instruction enables future mul instructions.
The don't() instruction disables future mul instructions.
Only the most recent do() or don't() instruction applies. At the beginning of the program, mul instructions are enabled.

For example:

`xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))`

This corrupted memory is similar to the example from before, but this time the mul(5,5) and mul(11,8) instructions are disabled because there is a don't() instruction before them. The other mul instructions function normally, including the one at the end that gets re-enabled by a do() instruction.

This time, the sum of the results is 48 (2*4 + 8*5).

Handle the new instructions; what do you get if you add up all of the results of just the enabled multiplications?



In [2]:

fun Iterable<Operator>.filterDisabled(): List<Operator.Mul> = buildList {
    var enabled = true
    this@filterDisabled.forEach {
        when(it) {
            is Operator.Do -> enabled = true
            is Operator.Dont -> enabled = false
            is Operator.Mul -> if (enabled) add(it)
        }
    }
}

fun calculatePart2(input: String): Int = parseOps(input)
    .filterDisabled()
    .sumOf { mul -> mul.a * mul.b }

val testInputPart2 = """xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"""

val testInputPart2Result = calculatePart2(testInputPart2)
println("Test Part 2: ${testInputPart2Result}")
assertEquals(48, testInputPart2Result)

val part2Result = calculatePart2(puzzleInput)
println("Part 2: ${part2Result}")
assertEquals(103811193, part2Result)

Test Part 2: 48
Part 2: 103811193
